In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# 1. Hyperparameter Tuning

Nastavovacie parametrov modelu == hyperparameter tuning

URL https://github.com/FIIT-IAU/IAU-2019-2020

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from sklearn import datasets

plt.rcParams['figure.figsize'] = 6, 4

iris = datasets.load_iris()
X = iris.data
y = iris.target
n_samples, n_features = X.shape
n_samples, n_features

#### Pridáme do dát 800 nových stĺpcov, ktoré sú ale úplne náhodné a nemali by nijak prispievať k úspešnosti. 

In [ ]:
# Trosku zasumime tie data, aby to vyzeralo krajsie
random_state = np.random.RandomState(0)
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]
X.shape

#### Skúsme teda natrénovať nejaký stromček

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

cls = DecisionTreeClassifier()
scores = cross_val_score(cls, X, y, cv=5, scoring = 'f1_macro')
scores.mean()

#### Dostali sme nejakú úspešnosť. Je to ale dobré číslo alebo nie? Potrebovali by sme si to overiť s niečím iným. Napríklad iným nastavením modelu.

In [ ]:
cls = DecisionTreeClassifier(max_depth=2)
scores = cross_val_score(cls, X, y, cv=5, scoring = 'f1_macro')
scores.mean()

Zlepšili sme to, ale nedalo by sa to zlepšiť ešte viac? Môžeme skúsiť ďalšie nastavenia, ale nechce sa nám to skúšať manuálne. Nedalo by sa to robiť nejak systematicky a hlavne automaticky?

## GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
clf = DecisionTreeClassifier

cv_params = {'max_depth': [1,2,3,4] + list(range(5,10,2)), 
             'criterion': ['gini', 'entropy'], 
             'min_samples_leaf': [1, 3] }
ind_params = {'random_state': 0}
optimization = GridSearchCV(clf(**ind_params), 
                            cv_params,
                            scoring = 'f1_macro', 
                            cv = 5, 
                            n_jobs = -1, 
                            verbose=True) 

#### GridSearch sa pokúsi zistiť úspešnosť na všetkých možných kombináciách parametrov, ktoré sme zadali do mriežky

In [ ]:
from sklearn.model_selection import ParameterGrid
print(len(ParameterGrid(cv_params)))
list(ParameterGrid(cv_params))

In [ ]:
%%time
optimization.fit(X, y)

In [ ]:
optimization.cv_results_ 

In [ ]:
list(filter(lambda x: 'best' in x, dir(optimization)))

In [ ]:
optimization.best_estimator_

In [ ]:
optimization.best_score_

## RandomSearch
- Alternatíva ku GridSearchu,
- Neprehľadávam systematicky celú mriežku, ale skúšam rôzne hodnoty parametrov náhodne.
- Obyčajne nájde riešenie veľmi **blízke** tomu najlepšiemu z GridSearchu na oveľa menej pokusov. Nemusí nájsť to najlepšie (ak by sa to najlepšie vôbec nachádzalo v mriežke).

In [ ]:
cv_params

In [ ]:
from scipy import stats

random_params = {
    'max_depth': stats.randint(1,10),
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': stats.randint(1,5)
}

Čo keby som namiesto vymenovania hodnôt dal náhodnú premennú?

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_optimization = RandomizedSearchCV(clf(**ind_params), 
                                         param_distributions = random_params, 
                                         n_iter = 10, 
                                         scoring = 'f1_macro', 
                                         cv = 5, 
                                         verbose=True, 
                                         random_state=42, 
                                         n_jobs = -1)

Skúsime spustiť optimalizáciu s menším počtom pokusov

In [ ]:
%%time
random_optimization.fit(X, y)

In [ ]:
random_optimization.best_estimator_

In [ ]:
random_optimization.best_score_

#### Viacero veľmi dobrých modelov má podobnú úspešnosť. Možno nám netreba nájsť ten úplne najlepší, ale rýchlo sa dostať k nejakému veľmi dobrému.

In [ ]:
sorted(optimization.cv_results_['mean_test_score'], reverse=True)